In [9]:
import numpy as np

from preprocessing import PRI_jet_num_split
from preprocessing import standardize, minmax_normalize
from preprocessing import clean_nan
from preprocessing import map_0_1, map_minus_1_1

from cross_validation import get_model, calculate_loss, accuracy, total_cross_validation

from implementations import build_poly

from scripts.helpers import load_csv_data
from scripts.helpers import predict_labels, create_csv_submission

from cross_validation import gamma_lambda_selection_cv
from cross_validation import plotting_graphs

In [10]:
def sort_arr(ids, y_pred):
    idx = ids.argsort()
    return ids[idx], y_pred[idx]

In [11]:
np.random.seed(1)

In [12]:
train_fname = "data/train.csv"
test_fname = "data/test.csv"
sumbission_fname = "data/submission.csv"

In [13]:
y_train, X_train, ids_train = load_csv_data(train_fname)
y_test, X_test, ids_test = load_csv_data(test_fname)

print("Shapes")
print(X_train.shape, y_train.shape, ids_train.shape)
print(X_test.shape, y_test.shape, ids_test.shape)
print()

Shapes
(250000, 30) (250000,) (250000,)
(568238, 30) (568238,) (568238,)



In [14]:
# combine_vals = True
combine_vals = False

train_subsets = PRI_jet_num_split(y_train, X_train, ids_train, combine_vals)
test_subsets = PRI_jet_num_split(y_test, X_test, ids_test, combine_vals)

print(f"Number of train subsets: { len(train_subsets) }")
print(f"Number of test subsets:  { len(test_subsets) }")
print()

assert len(train_subsets) == len(test_subsets)
num_subsets = len(train_subsets)

Number of train subsets: 6
Number of test subsets:  6



In [15]:
ids = np.array([])
y_pred = np.array([])

# Polynomial max degree
max_degree = 3

# GD step sizes, regularization factors
# lambdas = np.logspace(0, 1, 5) 
# gammas = np.logspace(0, 1, 5)
# gammas, lambdas = [1e-6, 1e-5, 1e-4], [0.0, 0.1, 1.0]

# getting the score by total cross_validation
testing_accuracy, testing_loss = [], []

In [16]:
for i in range(num_subsets):
    y_train_subset, X_train_subset, ids_train_subset = train_subsets[i]
    y_test_subset, X_test_subset, ids_test_subset = test_subsets[i]

    y_train_subset = map_0_1(y_train_subset)
    X_train_subset, X_test_subset = standardize(X_train_subset, X_test_subset)
    print(f"Train shape before feature expansion: {str(X_train_subset.shape):>12}   Test shape: {str(X_test_subset.shape):>12}")
#     X_train_subset, X_test_subset = build_poly(X_train_subset, max_degree), build_poly(X_test_subset, max_degree)
#     print(f"Train shape after  feature expansion: {str(X_train_subset.shape):>12}   Test shape: {str(X_test_subset.shape):>12}\n")

    N, D = X_train_subset.shape
    initial_w = np.random.randn(D)
    
    # tweak params
    k_fold = 4
    max_iters = 200
#     gammas, lambdas = [1e-5, 1e-4], [0.0, 1.0]
    gammas, lambdas = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1], [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0]
# overflow warning/error for lambda values more than 10 (need to check this)

    seed = 1
    batch_size = 1
    metric = 'CA'
    model = 'LOG_REG_GD'
    
    # gamma_lambda_selection_cv(y, tx, k_fold, initial_w, max_iters, gammas, lambdas, seed = 1, metric = 'CA', model = 'LOG_REG_GD')
    optimal_gamma, optimal_lambda_ = \
        gamma_lambda_selection_cv(y_train_subset, X_train_subset, k_fold, initial_w, max_iters, gammas, lambdas,
                                  seed = seed, batch_size = batch_size, metric = metric, model = model)
    
    print('Iter:', i, ' Best gamma:', optimal_gamma, ' Best lambda:', optimal_lambda_)
    
    
    # calling total cross_validation function to get tr_ac, te_ac, loss_tr, loss_te for each subset     
    loss_tr, loss_te, tr_ca, te_ca = total_cross_validation(y_train_subset, X_train_subset, k_fold, initial_w, \
                                max_iters, optimal_gamma, optimal_lambda_, seed = 1, batch_size = 1, model = 'LOG_REG_GD')
    testing_loss.append(loss_te)
    testing_accuracy.append(te_ca)

    
#      plotting graphs (between accuracy/loss and subset of hyperparameter values) 
#       will only plot for Reg_logistic after getting maximum accuracy on ai_crowd
#     plotting_graphs(y_train_subset, X_train_subset, k_fold, initial_w, max_iters, gammas, \
#                 lambdas, optimal_lambda_, optimal_gamma, i, seed = 1, model = 'LOG_REG_GD')
    
    
    # get_model(model, y, tx, initial_w, max_iters, gamma, lambda_, batch_size)
    w = get_model(model, y_train_subset, X_train_subset, initial_w, max_iters, optimal_gamma, optimal_lambda_, batch_size)

    y_pred_test = np.array(map_minus_1_1(predict_labels(w, X_test_subset)))

    ids = np.concatenate((ids, ids_test_subset))
    y_pred = np.concatenate((y_pred, y_pred_test))

ids, y_pred = sort_arr(ids, y_pred)
create_csv_submission(ids, y_pred, sumbission_fname)


# ca_te, loss_te of the whole model
testing_accuracy = sum(testing_accuracy)/float(num_subsets)
testing_loss = sum(testing_loss)/float(num_subsets)

print ('total_ca_te:', testing_accuracy, 'total_loss_te:', testing_loss)

# the below example run is when combined=True.

Train shape before feature expansion:  (73790, 18)   Test shape: (168195, 18)
(0, 0)/(7, 7)
(0, 1)/(7, 7)
(0, 2)/(7, 7)
(0, 3)/(7, 7)
(0, 4)/(7, 7)
(0, 5)/(7, 7)
(0, 6)/(7, 7)
(1, 0)/(7, 7)
(1, 1)/(7, 7)
(1, 2)/(7, 7)
(1, 3)/(7, 7)
(1, 4)/(7, 7)
(1, 5)/(7, 7)
(1, 6)/(7, 7)
(2, 0)/(7, 7)
(2, 1)/(7, 7)
(2, 2)/(7, 7)
(2, 3)/(7, 7)
(2, 4)/(7, 7)
(2, 5)/(7, 7)
(2, 6)/(7, 7)
(3, 0)/(7, 7)
(3, 1)/(7, 7)
(3, 2)/(7, 7)
(3, 3)/(7, 7)
(3, 4)/(7, 7)
(3, 5)/(7, 7)
(3, 6)/(7, 7)
(4, 0)/(7, 7)
(4, 1)/(7, 7)
(4, 2)/(7, 7)
(4, 3)/(7, 7)
(4, 4)/(7, 7)
(4, 5)/(7, 7)
(4, 6)/(7, 7)
(5, 0)/(7, 7)
(5, 1)/(7, 7)
(5, 2)/(7, 7)
(5, 3)/(7, 7)
(5, 4)/(7, 7)
(5, 5)/(7, 7)
(5, 6)/(7, 7)
(6, 0)/(7, 7)
(6, 1)/(7, 7)
(6, 2)/(7, 7)
(6, 3)/(7, 7)
(6, 4)/(7, 7)
(6, 5)/(7, 7)
(6, 6)/(7, 7)
CA_te:
 [[0.74348132 0.74348132 0.74348132 0.74348132 0.74348132 0.74348132
  0.74348132]
 [0.7775248  0.7775248  0.7775248  0.7775248  0.7775248  0.7775248
  0.77755191]
 [0.70633436 0.70633436 0.70633436 0.70633436 0.70633436 0.706334

Train shape before feature expansion:  (68114, 29)   Test shape: (155460, 29)
(0, 0)/(7, 7)
(0, 1)/(7, 7)
(0, 2)/(7, 7)
(0, 3)/(7, 7)
(0, 4)/(7, 7)
(0, 5)/(7, 7)
(0, 6)/(7, 7)
(1, 0)/(7, 7)
(1, 1)/(7, 7)
(1, 2)/(7, 7)
(1, 3)/(7, 7)
(1, 4)/(7, 7)
(1, 5)/(7, 7)
(1, 6)/(7, 7)
(2, 0)/(7, 7)
(2, 1)/(7, 7)
(2, 2)/(7, 7)
(2, 3)/(7, 7)
(2, 4)/(7, 7)
(2, 5)/(7, 7)
(2, 6)/(7, 7)
(3, 0)/(7, 7)
(3, 1)/(7, 7)
(3, 2)/(7, 7)
(3, 3)/(7, 7)
(3, 4)/(7, 7)
(3, 5)/(7, 7)
(3, 6)/(7, 7)
(4, 0)/(7, 7)
(4, 1)/(7, 7)
(4, 2)/(7, 7)
(4, 3)/(7, 7)
(4, 4)/(7, 7)
(4, 5)/(7, 7)
(4, 6)/(7, 7)
(5, 0)/(7, 7)
(5, 1)/(7, 7)
(5, 2)/(7, 7)
(5, 3)/(7, 7)
(5, 4)/(7, 7)
(5, 5)/(7, 7)
(5, 6)/(7, 7)
(6, 0)/(7, 7)
(6, 1)/(7, 7)
(6, 2)/(7, 7)
(6, 3)/(7, 7)
(6, 4)/(7, 7)
(6, 5)/(7, 7)
(6, 6)/(7, 7)
CA_te:
 [[0.66433228 0.66433228 0.66433228 0.66433228 0.66431759 0.66433228
  0.66434696]
 [0.71827284 0.71827284 0.71827284 0.71827284 0.71827284 0.71827284
  0.71828753]
 [0.65744656 0.65744656 0.65744656 0.65744656 0.65746124 0.65744

In [ ]:
# USE THIS TO TEST 1 SUBSET

i = 0
y_train_subset, X_train_subset, ids_train_subset = train_subsets[i]
y_test_subset, X_test_subset, ids_test_subset = test_subsets[i]

y_train_subset = map_0_1(y_train_subset)
X_train_subset, X_test_subset = standardize(X_train_subset, X_test_subset)
print(f"Train shape before feature expansion: {str(X_train_subset.shape):>12}   Test shape: {str(X_test_subset.shape):>12}\n")
X_train_subset, X_test_subset = build_poly(X_train_subset, max_degree), build_poly(X_test_subset, max_degree)
print(f"Train shape after  feature expansion: {str(X_train_subset.shape):>12}   Test shape: {str(X_test_subset.shape):>12}\n")

N, D = X_train_subset.shape
initial_w = np.random.randn(D)

In [ ]:
# need to choose optimal lambda and optimal gamma together
k_fold = 5 # can experiment with different numbers
max_iters = 50

# GD step sizes, regularization factors
gammas, lambdas = [1e-5, 1], [0.0, 0.001]
optimal_gamma, optimal_lambda_ = gamma_lambda_selection_cv(y_train_subset, X_train_subset, k_fold, initial_w, max_iters, gammas, lambdas)

print(optimal_gamma, optimal_lambda_)

_, w = reg_logistic_regression(y_train_subset, X_train_subset, optimal_lambda_, initial_w, max_iters, optimal_gamma)
y_pred_test = np.array(map_minus_1_1(predict_labels(w, X_test_subset)))

ids = np.concatenate((ids, ids_test_subset))
y_pred = np.concatenate((y_pred, y_pred_test))

ids, y_pred = sort_arr(ids, y_pred)